In [142]:
import parse_data.preprocess as preprocess
import parse_data.loading as loading
from analysis.response_times import calculate_response_times, plot_response_times
import globals
from plotting import plot_octagon, plot_trajectory
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib as mpl
import math

In [143]:
# paths
# data_folder = '/home/tom/Documents/SWC/data' # desktop Duan Lab
# json_filename = '240913_Yansu_Jerry/2024-09-13_11-53-34_YansuSecondSolo.json' 
# json_filename = '240913_Yansu_Jerry/2024-09-13_11-31-00_YansuJerrySocial.json'
data_folder = r'D:\Users\Tom\OneDrive\PhD\SWC\data' # desktop home
# json_filename = r'first_experiments_2409\240913\2024-09-13_11-23-37_YansuFirstSolo.json'
json_filename = r'first_experiments_2409\240913\2024-09-13_11-31-00_YansuJerrySocial.json'


In [144]:
# Load JSON file into pandas df with collapsed data dictionary and adjustments based on date of recording 
df = loading.loading_pipeline(data_folder, json_filename)

Data is from period before 2024-09-13 11:31:00
Running dataframe through playerinfo_playerposition_conversion.
Loading complete.


In [145]:
# Pre-process data 
df = preprocess.standard_preprocessing(df)

Preprocessing complete.


In [146]:
## Extract single trials ##

In [147]:
def split_session_by_trial(df):
    
    # groupby produces an interable of tuples with the group key and the dataframe 
    trials_list = [data for _, data in df.groupby('data.trialNum')]
    
    # exclude trial 0 (could also exclude trial 1)
    trials_list = trials_list[1:]

    # if final  trial does not contain a server selected trigger activation, discard it
    if not globals.SELECTED_TRIGGER_ACTIVATION in trials_list[-1]['eventDescription'].unique():
        trials_list = trials_list[:-1]

    return trials_list


In [148]:
trials_list = split_session_by_trial(df)

In [149]:
## rotate trials ##

In [150]:
def find_rotation_angle_trial(trial):
    """ Find CCW angle of rotation for vector to 
    rotate arena s.t. high wall is at wall 1"""

    
    
    # identify trial walls
    wall1 = trial.iloc[0]['data.wall1']
    wall2 = trial.iloc[0]['data.wall2']
    
    # find difference of high wall to wall 1
    difference = wall1 - 1
        
    # find CCW rotation angle 
    unitary_rotation_ang = 2*math.pi/globals.NUM_WALLS
    theta = unitary_rotation_ang * difference

    return theta

In [151]:
def rotation(trial, theta):
    """ Take counterclockwise rotation angle and starting vector
        Return rotated vector """

    rotated_coordinates = []

    num_players = preprocess.num_players(df)
    
    trial_copy = trial.copy()

    # create rotation matrix
    rotM = np.array([
                    [math.cos(theta), -math.sin(theta)],
                     [math.sin(theta), math.cos(theta)]
                    ])

    for i in range(num_players):
        x,y = trial_copy[globals.PLAYER_LOC_DICT[i]['xloc']], trial_copy[globals.PLAYER_LOC_DICT[i]['yloc']]

        this_coordinates = np.vstack([x,y])
        this_rotated_coordinates = np.matmul(rotM, this_coordinates) 

        rotated_coordinates.append(this_rotated_coordinates)

    return rotated_coordinates
    
        
    

    
        
    

In [152]:
def placeholder(trial, rotated_coordinates):
    trial_copy = trial.copy()

    for i in range(len(rotated_coordinates)):
        trial_copy[globals.PLAYER_LOC_DICT[i]['xloc']] = rotated_coordinates[i][0]
        trial_copy[globals.PLAYER_LOC_DICT[i]['yloc']] = rotated_coordinates[i][1]

    return trial_copy
        


In [153]:
trial_idx = 84

In [154]:
trial = trials_list[trial_idx]

In [155]:
trial.iloc[0]

timeLocal                           1900-01-01 11:45:36.425000
timeApplication                         0 days 00:15:32.356000
eventDescription                                   trial start
data.playerPosition.0.clientId                             NaN
data.playerPosition.0.location.x                   3.909650087
data.playerPosition.0.location.y                   1.984999657
data.playerPosition.0.location.z                  -0.144384339
data.playerPosition.0.rotation.x                   9.736743927
data.playerPosition.0.rotation.y                  46.491825104
data.playerPosition.0.rotation.z                  -0.000000217
data.playerPosition.1.clientId                             NaN
data.playerPosition.1.location.x                           NaN
data.playerPosition.1.location.y                           NaN
data.playerPosition.1.location.z                           NaN
data.playerPosition.1.rotation.x                           NaN
data.playerPosition.1.rotation.y                       

In [156]:
theta = find_rotation_angle_trial(trial)

In [157]:
rotated_coords = rotation(trial, theta)

In [158]:
    rotated_coords = np.array(rotated_coords)